<a href="https://colab.research.google.com/github/Kimjibeom/Project_Book/blob/main/%EC%9E%85%EB%A0%A5_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B1%85_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive 마운트 하기





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/')

# 키워드 콘텐츠 필터링을 이용한 도서 추천

In [45]:
import pandas as pd
import numpy as np
from ast import literal_eval

book = pd.read_csv('/content/drive/MyDrive/data/book_data_nouns.csv')

# 단어 벡터화

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

# ngram_range=(1, 2) : 단어를 1개 혹은 2개 연속으로 보겠다
tfidf_vec = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vec.fit_transform(book['keywords'])

* keyword 데이터를 TfidfVectorizer(TFIDF 기반 벡터화) 메소드로 벡터화한다.
* TF-IDF 방식으로 단어의 가중치를 조정한 BOW 벡터를 만든다.
* ngram_range는 단어를 1개 혹은 2개 연속으로 보고 이를 하나로 묶는다는 의미이다.
* 그러한 TfidfVectorizer를 기반으로 keyword 데이터를 바로 fit 및 transfrom 한다.

# 유사도 측정

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

# 개의 도서랑 개의 도서끼리 유사도를 구함
# 자신과의 유사도는 1
genres_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(genres_similarity)
# 유사도 값이 높은 영화의 제목
# 유사도 값이 높은 순으로 인덱스 값을 뽑아낸다
similar_index = np.argsort(-genres_similarity)
print(similar_index)

[[1.         0.00438686 0.         ... 0.00125905 0.00262927 0.        ]
 [0.00438686 1.         0.         ... 0.00705829 0.00378669 0.0128543 ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.00125905 0.00705829 0.         ... 1.         0.03222907 0.01013079]
 [0.00262927 0.00378669 0.         ... 0.03222907 1.         0.00656322]
 [0.         0.0128543  0.         ... 0.01013079 0.00656322 1.        ]]
[[    0 10133  5882 ...  4518  4534 10733]
 [    1  3746  9429 ...  6421  2151  5366]
 [    2  5151  8869 ...  4274  4340 10733]
 ...
 [10731  8018  7323 ...  3924  3806  3540]
 [10732  2940  2647 ...  6608  6577  5366]
 [10733   268   267 ...  5185  5245  5366]]


* cosine_similarity 메소드는 코사인 유사도를 기반으로 유사도를 측정해준다.
* 측정된 유사도 행렬은 도서개수x도서개수의 shape를 가진다.
* np.argsort(-genres_similarity)를 수행하면 genres_similarity를 유사도가 높은 순서대로 정렬한 인덱스 numpy 배열을 리턴한다. 즉, 각 도서에 해당하는 행마다 자신과 유사한 도서를 순서대로 가지고 있는 배열이 생성된다.

# 사용자입력 및 결과 출력

In [58]:
input_keywords = input()

book_index = book[book['keywords'].str.contains(input_keywords)].index
similar_book = similar_index[book_index, :5]

# 인덱스로 사용하기 용이하도록 1차원으로 변경
similar_book_index = similar_book.reshape(-1)

result = book.iloc[similar_book_index]
result = result.head()
print(result)

사회
                          id  ...                                           keywords
2     11931111_9788963527192  ...                      인간 행동 사회 환경 이론 인간 행동 사회 환경 학습
5151  18059136_9791191109252  ...  최근 회복 대학교육 협의 세한 사회 복지 교과 개정 지침 인간 행동 영향 사회 환경...
8869  20642492_9791163283164  ...  사회 복지사 환경 인간 관점 인간 행동 개인 부분 환경 영향 전제 인간 인간 행동 ...
260   16357500_9791137206120  ...  그간 대학 대학원 인간 행동 사회 환경 다년 강의 김해영 교수 접근 주제 조직 인간...
8151  20578014_9788999723247  ...  권중돈 여름 경남 의령 동네 숭실대 학교 영어 영문학 전공 사회사업 전공 연세대학교...

[5 rows x 11 columns]


* 사용자가 입력한 도서의 인덱스 값을 찾아내고 similar_index 에 접근한다.
* similar_index에서 얻어낸 정보를 기반으로 유사도가 제일 가까운 5개의 영화 인덱스를 가져온다.
* 해당 데이터는 2차원으로 되어 있는데, 인덱스를 활용하기 용이하도록 1차원으로 변경한다.
* book 데이터에서 해당 인덱스 정보를 활용해 값을 출력한다.